In [30]:
import os
import sys
import pickle
import pandas as pd
import numpy as np
import talib as ta
from datetime import datetime, timedelta
#%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image, display_png
os.chdir('/Users/shun/PycharmProjects/crypto-onibot')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 1000)

In [31]:
executions = pd.read_csv('collect/executions/000000', names=['timestamp', 'datetime', 'side', 'price', 'amount'])

header = ['timestamp']
for i in range(50):
    for col in ['side', 'amount', 'price']:
        header.append(col + '{0:02d}'.format(i))
orderbook = pd.read_csv('collect/orderbook/000000', names=header)

inago = pd.read_csv('collect/inago/inago.csv', sep='\t', names=['id', 'board_name', 'taker_side', 'volume', 'last_price', 'pair_currency', 'from_unix_time', 'to_unix_time', 'from_datetime', 'to_datetime', 'timestamp'])

In [35]:
#executions = executions.iloc[:1000]
#orderbook = orderbook.iloc[:1000]


dtype('<M8[ns]')

In [37]:
executions.index = pd.to_datetime(executions.timestamp.apply(lambda x: datetime.fromtimestamp(x/1000)))
# freq format: https://stackoverflow.com/questions/35339139/where-is-the-documentation-on-pandas-freq-tags
executions = executions.groupby(pd.TimeGrouper(freq='S')).last()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
orderbook.index = orderbook.timestamp.apply(lambda x: datetime.fromtimestamp(x/1000))
# freq format: https://stackoverflow.com/questions/35339139/where-is-the-documentation-on-pandas-freq-tags
orderbook = orderbook.groupby(pd.TimeGrouper(freq='S')).last()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
def format_orderbook(orderbook, executions):
    '''
    約定時刻付近の板のみ抽出する
    '''   
    # 高速化のためにnumpyにする
    execution_timestamp = executions.timestamp.values
    orderbook_timestamp = orderbook.timestamp.values
    orderbook_val = orderbook.values
    
    formatted_orderbook = [[''] * orderbook.shape[1]]
    for i in range(1, executions.shape[0]):
        # 前回の約定時刻~今の約定時刻の板を取得
        tmp = orderbook_val[(execution_timestamp[i - 1] < orderbook_timestamp) & (orderbook_timestamp <=  execution_timestamp[i])].tolist()        
        if len(tmp) > 0:
            formatted_orderbook.append(tmp[-1])
        else:
            # 条件に合う板がなければ前回の板を追加
            formatted_orderbook.append(formatted_orderbook[i - 1])
    return pd.DataFrame(formatted_orderbook, columns=orderbook.columns)    

In [7]:
def format_inago(inago, executions):
    '''
    約定時刻付近のイナゴのみ抽出する
    '''   
    # 高速化のためにnumpyにする
    execution_timestamp = executions.timestamp.values
    inago_timestamp = inago.timestamp.values
    inago_val = inago.values
    
    formatted_inago = [[''] * inago.shape[1]]
    for i in range(1, executions.shape[0]):
        # 前回の約定時刻~今の約定時刻のイナゴを取得
        tmp = inago_val[(execution_timestamp[i - 1] < inago_timestamp) & (inago_timestamp <=  execution_timestamp[i])].tolist()        
        if len(tmp) > 0:
            formatted_inago.append(tmp[-1])
        else:
            # 条件に合う板がなければ空データを追加
            formatted_inago.append([''] * inago.shape[1])
    return pd.DataFrame(formatted_inago, columns=inago.columns)

In [8]:
import time
start = time.time()
formatted_orderbook = format_orderbook(orderbook, executions)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less_equal
  del sys.path[0]


elapsed_time:0.11576700210571289[sec]


In [9]:
formatted_orderbook.loc[formatted_orderbook['timestamp'] != ''].head()

,timestamp,side00,amount00,price00,side01,amount01,price01,side02,amount02,price02,side03,amount03,price03,side04,amount04,price04,side05,amount05,price05,side06,amount06,price06,side07,amount07,price07,side08,amount08,price08,side09,amount09,price09,side10,amount10,price10,side11,amount11,price11,side12,amount12,price12,side13,amount13,price13,side14,amount14,price14,side15,amount15,price15,side16,amount16,price16,side17,amount17,price17,side18,amount18,price18,side19,amount19,price19,side20,amount20,price20,side21,amount21,price21,side22,amount22,price22,side23,amount23,price23,side24,amount24,price24,side25,amount25,price25,side26,amount26,price26,side27,amount27,price27,side28,amount28,price28,side29,amount29,price29,side30,amount30,price30,side31,amount31,price31,side32,amount32,price32,side33,amount33,price33,side34,amount34,price34,side35,amount35,price35,side36,amount36,price36,side37,amount37,price37,side38,amount38,price38,side39,amount39,price39,side40,amount40,price40,side41,amount41,price41,side42,amount42,price42,side43,amount43,price43,side44,amount44,price44,side45,amount45,price45,side46,amount46,price46,side47,amount47,price47,side48,amount48,price48,side49,amount49,price49
74,1.55315e+12,Sell,327440,4031,Sell,145813,4030.5,Sell,574591,4030,Sell,306204,4029.5,Sell,265783,4029,Sell,52870,4028.5,Sell,425262,4028,Sell,115671,4027.5,Sell,1.96507e+06,4027,Sell,371743,4026.5,Sell,522498,4026,Sell,388125,4025.5,Sell,537321,4025,Sell,341846,4024.5,Sell,212467,4024,Sell,122873,4023.5,Sell,130981,4023,Sell,137877,4022.5,Sell,290527,4022,Sell,141541,4021.5,Sell,34662,4021,Sell,58250,4020.5,Sell,198772,4020,Sell,115646,4019.5,Sell,3.84588e+06,4019,Buy,1.42777e+06,4018.5,Buy,713201,4018,Buy,296939,4017.5,Buy,337586,4017,Buy,245076,4016.5,Buy,319270,4016,Buy,300633,4015.5,Buy,691611,4015,Buy,333980,4014.5,Buy,706097,4014,Buy,744558,4013.5,Buy,841063,4013,Buy,377758,4012.5,Buy,700364,4012,Buy,471301,4011.5,Buy,540272,4011,Buy,486565,4010.5,Buy,637586,4010,Buy,50738,4009.5,Buy,313351,4009,Buy,237438,4008.5,Buy,290255,4008,Buy,284600,4007.5,Buy,455493,4007,Buy,153863,4006.5
75,1.55315e+12,Sell,327440,4031,Sell,145813,4030.5,Sell,574591,4030,Sell,306204,4029.5,Sell,265783,4029,Sell,52870,4028.5,Sell,425262,4028,Sell,115671,4027.5,Sell,1.96507e+06,4027,Sell,371743,4026.5,Sell,522498,4026,Sell,388125,4025.5,Sell,537321,4025,Sell,341846,4024.5,Sell,212467,4024,Sell,122873,4023.5,Sell,130981,4023,Sell,137877,4022.5,Sell,290527,4022,Sell,141541,4021.5,Sell,34662,4021,Sell,58250,4020.5,Sell,198772,4020,Sell,115646,4019.5,Sell,3.84588e+06,4019,Buy,1.42777e+06,4018.5,Buy,713201,4018,Buy,296939,4017.5,Buy,337586,4017,Buy,245076,4016.5,Buy,319270,4016,Buy,300633,4015.5,Buy,691611,4015,Buy,333980,4014.5,Buy,706097,4014,Buy,744558,4013.5,Buy,841063,4013,Buy,377758,4012.5,Buy,700364,4012,Buy,471301,4011.5,Buy,540272,4011,Buy,486565,4010.5,Buy,637586,4010,Buy,50738,4009.5,Buy,313351,4009,Buy,237438,4008.5,Buy,290255,4008,Buy,284600,4007.5,Buy,455493,4007,Buy,153863,4006.5
76,1.55315e+12,Sell,327440,4031,Sell,145813,4030.5,Sell,574591,4030,Sell,306204,4029.5,Sell,265783,4029,Sell,52870,4028.5,Sell,425262,4028,Sell,115671,4027.5,Sell,1.96507e+06,4027,Sell,371743,4026.5,Sell,522498,4026,Sell,388125,4025.5,Sell,537321,4025,Sell,341846,4024.5,Sell,212467,4024,Sell,122873,4023.5,Sell,130981,4023,Sell,137877,4022.5,Sell,290527,4022,Sell,141541,4021.5,Sell,34662,4021,Sell,58250,4020.5,Sell,198772,4020,Sell,115646,4019.5,Sell,3.84588e+06,4019,Buy,1.42777e+06,4018.5,Buy,713201,4018,Buy,296939,4017.5,Buy,337586,4017,Buy,245076,4016.5,Buy,319270,4016,Buy,300633,4015.5,Buy,691611,4015,Buy,333980,4014.5,Buy,706097,4014,Buy,744558,4013.5,Buy,841063,4013,Buy,377758,4012.5,Buy,700364,4012,Buy,471301,4011.5,Buy,540272,4011,Buy,486565,4010.5,Buy,637586,4010,Buy,50738,4009.5,Buy,313351,4009,Buy,237438,4008.5,Buy,290255,4008,Buy,284600,4007.5,Buy,455493,4007,Buy,153863,4006.5
77,1.55315e+12,Sell,327440,4031,Sell,145813,4030.5,Sell,574591,4030,Sell,306204,4029.5

In [10]:
import time
start = time.time()
formatted_inago = format_inago(inago, executions)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:0.0843508243560791[sec]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less_equal
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less
  del sys.path[0]


In [11]:
formatted_inago.loc[formatted_inago['timestamp'] != ''].head()

,id,board_name,taker_side,volume,last_price,pair_currency,from_unix_time,to_unix_time,from_datetime,to_datetime,timestamp
485,4,BitMEX_XBTUSD,sell,205,4017,USD,1553146553000,1553146572000,2019-03-21 14:35:53,2019-03-21 14:36:12,1553146572300
1642,6,BitMEX_XBTUSD,sell,272,4017.5,USD,1553147724000,1553147729000,2019-03-21 14:55:24,2019-03-21 14:55:29,1553147729719
3489,9,BitMEX_XBTUSD,buy,220,4019,USD,1553149557000,1553149575000,2019-03-21 15:25:57,2019-03-21 15:26:15,1553149575977
3503,10,BitMEX_XBTUSD,buy,201,4019.5,USD,1553149575000,1553149590000,2019-03-21 15:26:15,2019-03-21 15:26:30,1553149590131
3524,11,BitMEX_XBTUSD,buy,202,4020.5,USD,1553149590000,1553149610000,2019-03-21 15:26:30,2019-03-21 15:26:50,1553149610445


In [38]:
# 約定履歴からohlcvを作成
executions_bids = executions.loc[executions.side == 'Sell']
ohlc = executions_bids.price.resample('S').ohlc()